In [30]:
import pandas as pd
import random
from simpletransformers.classification import MultiLabelClassificationModel
from sklearn.model_selection import train_test_split
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from scoping.models import *

import matplotlib.pyplot as plt



In [ ]:
model = MultiLabelClassificationModel(
    'roberta', 'roberta-base',
    use_cuda=False,
    num_labels=4, args={
        'train_batch_size':2, 'gradient_accumulation_steps':16, 
        'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512
    })

In [2]:
qs = Query.objects.filter(pk__in=[8145,7368,8272])
physical_tags = Tag.objects.filter(query__in=qs,id__gt=5177)
pdos = DocOwnership.objects.filter(tag__in=physical_tags,relevant__gt=0).values_list('doc__id',flat=True)
df = pd.read_csv('../data/category_predictions.csv')
df = df[df['id'].isin(list(pdos))]
df = df[df['relevant']==1]
print(df.shape)
df.head()
print(df.shape)
df.head()

(682, 235)
(682, 235)


,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,12 - Coastal and marine Ecosystems,12 - Human and managed,...,"12 - Mountains, snow and ice - k_prediction",12 - Human and managed - k_prediction,2 - 2.1. Climate change attribution - k_prediction,2 - 2.2 Trend attribution - k_prediction,2 - 2.4. Sensitivity - k_prediction,2 - 2.5. Detection of a regional climate trend (no attribution) - k_prediction,2 - 2.1. Climate change attribution - prediction,2 - 2.2 Trend attribution - prediction,2 - 2.4. Sensitivity - prediction,2 - 2.5. Detection of a regional climate trend (no attribution) - prediction
7,899880,A new fire policy reinforcing aggressive fire ...,Wildfire Policy in Mediterranean France: How F...,Fire paradox; fire policy; fire prevention; fi...,"['Public, Environmental & Occupational Health;...",0.0,1.0,1.0,0.0,0.0,...,0.012158,0.007934,0.056233,0.451148,0.101784,0.015225,NaN,NaN,NaN,NaN
10,617231,Although it is well established that butterfly...,Determinants of species richness in generalist...,0,['Biodiversity Conservation; Ecology'],0.0,1.0,1.0,0.0,0.0,...,0.011314,0.018131,0.049645,0.459345,0.187517,0.016066,NaN,NaN,NaN,NaN
49,670400,Heat stress impacts on crop growth and yield h...,Heat stress impacts on wheat growth and yield ...,Adaptation; Crop production; Cultivars; Extrem...,['Agronomy'],0.0,1.0,1.0,0.0,1.0,...,0.014771,0.964162,0.064188,0.319689,0.394913,0.036509,NaN,NaN,NaN,NaN
53,572040,PREMISE OF THE STUDY : Mountainous regions sup...,Long-term shifts in the phenology of rare and ...,Colorado; climate change; endangered and imper...,['Plant Sciences'],0.0,1.0,1.0,0.0,0.0,...,0.010197,0.030651,0.048232,0.292799,0.394691,0.031180,NaN,NaN,NaN,NaN
79,101763,Background: Heat is recognized as one of the d...,Heat and Mortality in New York City Since the ...,0,"['Public, Environmental & Occupational Health']",0.0,1.0,1.0,0.0,1.0,...,0.023784,0.984619,0.063218,0.194151,0.500000,0.028288,NaN,NaN,NaN,NaN


In [3]:
cols = []
for c in df.columns:
    if "2 - 2." in c and "hidden" not in c and "k_prediction" not in c:
        print(c)
        print(df[c].sum())
        if df[c].sum() > 45:
            df[c] = df[c].astype(int)
            cols.append(c)
cols

2 - 2.1. Climate change attribution
46.0
2 - 2.2 Trend attribution
277.0
2 - 2.3. Attribution to extreme event
26.0
2 - 2.4. Sensitivity
251.0
2 - 2.5. Detection of a regional climate trend (no attribution)
66.0
2 - 2.6. Null results
16.0
2 - 2.1. Climate change attribution - prediction
0.0
2 - 2.2 Trend attribution - prediction
0.0
2 - 2.4. Sensitivity - prediction
0.0
2 - 2.5. Detection of a regional climate trend (no attribution) - prediction
0.0


['2 - 2.1. Climate change attribution',
 '2 - 2.2 Trend attribution',
 '2 - 2.4. Sensitivity',
 '2 - 2.5. Detection of a regional climate trend (no attribution)']

In [4]:
df['labels'] = list(df[cols].values)
df['text'] = df['content']
df = df[['text','labels']]
df.head()

,text,labels
7,A new fire policy reinforcing aggressive fire ...,"[0, 1, 1, 0]"
10,Although it is well established that butterfly...,"[0, 0, 1, 0]"
49,Heat stress impacts on crop growth and yield h...,"[0, 0, 1, 0]"
53,PREMISE OF THE STUDY : Mountainous regions sup...,"[0, 1, 0, 0]"
79,Background: Heat is recognized as one of the d...,"[0, 1, 0, 0]"


In [5]:
train_df, test_df = train_test_split(df)
print(train_df.shape)
print(test_df.shape)

(511, 2)
(171, 2)


In [6]:
# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

Running loss: 0.380529


Running loss: 0.490360


Running loss: 0.328565



In [7]:
result

{'LRAP': 0.7748538011695905, 'eval_loss': 0.46069562570615247}

In [22]:
y = np.vstack(list(test_df['labels']))
print(y.shape)
for j in range(y.shape[1]):
    print(j)

(171, 4)
0
1
2
3


In [35]:
np.argmax(model_outputs,axis=1)
         

array([1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1,
       2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2,
       2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2,
       2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1])

In [25]:
y[:,2]

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1])